# Queueing Model

reference 

https://docs.python.org/3/library/queue.html

https://numpy.org/doc/stable/reference/random/generated/numpy.random.exponential.html

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import time
from collections import deque
from collections import namedtuple as nt


In [2]:
# import the math model 
%run ../Wait_Times_Math_Model/QueueWaitTimes.ipynb

Transaction Count:  3389521
Mean Block Size:  2266.0
Fee Rate Priority Range:  [52380.95238095238, 30.175438596491237, 18.529411764705888, 5.0, 0.0]
Priority Group 1
Transaction Count: 2035967, Raw Wait Time (Mean): 2474.0209728350214
Feerate Min: 30.175580221997986, Max: 52380.95238095238
Lambda: 1.851985231178986
Priority Group 2
Transaction Count: 565691, Raw Wait Time (Mean): 6085.267361510082
Feerate Min: 18.52950075642965, Max: 30.175438596491237
Lambda: 0.5145718852078014
Priority Group 3
Transaction Count: 385863, Raw Wait Time (Mean): 9987.642969136714
Feerate Min: 5.000028548752705, Max: 18.529411764705888
Lambda: 0.3509941847085032
Priority Group 4
Transaction Count: 402000, Raw Wait Time (Mean): 11979.932554726369
Feerate Min: 0.907563025210084, Max: 5.0
Lambda: 0.36567295193583804

Sum Of All Lambda:  3.083224253031129
Mu:  0.0016666666666666668
Priority Group 1
My Z: 0.9992742750913941, SciPy Z: 0.9992742751058932
Priority Group 2
My Z: 0.9995497987137135, SciPy Z: 0.9995

## Data Loading 

In [3]:
# load the data 
data1 = pd.read_csv("TimetxinBlock621500.csv")
data1.columns=['index','inputs','outputs','trans_version','trans_size','trans_weight','received_time','relay_node','locktime','trans_fee','confirmed_block_height','index_block_height','confirm_time','waiting_time','feerate','enter_block_height','waiting_block_num','valid_time','valid_block_height','valid_waiting','last_block_interval']
data2 = pd.read_csv("TimetxinBlock622000.csv")
data2.columns=['index','inputs','outputs','trans_version','trans_size','trans_weight','received_time','relay_node','locktime','trans_fee','confirmed_block_height','index_block_height','confirm_time','waiting_time','feerate','enter_block_height','waiting_block_num','valid_time','valid_block_height','valid_waiting','last_block_interval']
data3 = pd.read_csv("TimetxinBlock622500.csv")
data3.columns=['index','inputs','outputs','trans_version','trans_size','trans_weight','received_time','relay_node','locktime','trans_fee','confirmed_block_height','index_block_height','confirm_time','waiting_time','feerate','enter_block_height','waiting_block_num','valid_time','valid_block_height','valid_waiting','last_block_interval']


# combine the data into one dataframe
data = pd.concat([data1, data2, data3], ignore_index = True)
# data1.columns=['index','inputs','outputs','trans_version','trans_size','trans_weight','received_time','relay_node','locktime','trans_fee','confirmed_block_height','index_block_height','confirm_time','waiting_time','feerate','enter_block_height','waiting_block_num','valid_time','valid_block_height','valid_waiting','last_block_interval']
print(data.shape)
data = data.sort_values(by=['received_time'])
data.head(20)



(3389593, 21)


,index,inputs,outputs,trans_version,trans_size,trans_weight,received_time,relay_node,locktime,trans_fee,...,index_block_height,confirm_time,waiting_time,feerate,enter_block_height,waiting_block_num,valid_time,valid_block_height,valid_waiting,last_block_interval
68058,0,4,1,2,1220,2258,1583756823,0,620940,567,...,0,1583810262,53439,1.004429,620943,89,1.583757e+09,620943,53439.0,26.0
67840,0,1,2,1,284,806,1583756865,0,0,202,...,0,1583810262,53397,1.002481,620943,89,1.583757e+09,620943,53397.0,68.0
68123,0,3,1,2,926,1736,1583756869,0,620940,436,...,0,1583810262,53393,1.004608,620943,89,1.583757e+09,620943,53393.0,72.0
16125,0,2,2,2,420,1032,1583756885,0,620942,258,...,0,1583796952,40067,1.000000,620943,64,1.583757e+09,620943,40067.0,88.0
10294,0,2,2,2,420,1032,1583756899,0,620887,258,...,0,1583796307,39408,1.000000,620943,62,1.583757e+09,620943,39408.0,102.0
67964,0,1,2,2,247,661,1583756901,0,620942,166,...,0,1583810262,53361,1.004539,620943,89,1.583757e+09,620943,53361.0,104.0
68118,0,4,2,1,667,2668,1583756912,0,0,670,...,0,1583810262,53350,1.004498,620943,89,1.583757e+09,620943,53350.0,115.0
67558,0,1,2,1,283,805,1583756927,0,0,202,...,0,1583810262,53335,1.003727,620943,89,1.583757e+09,620943,53335.0,130.0
67653,0,1,2,2,247,661,1583756972,0,620942,166,...,0,1583810262,53290,1.004539,620943,89,1.583757e+09,620943,53290.0,175.0
67866,0,1,2,1,283,805,1583757002,0,0,202,...,0,1583810262,53260,1.003727,620943,89,1.583757e+09,620943,53260.0,205.0


## Parameters - User inputs

In [4]:
# Default block size if not specified 
block_groups = data.groupby(['confirmed_block_height'])['confirmed_block_height'].count()
mean_block_size = float(round(block_groups.mean()))

# Replace with user input
b=mean_block_size
print("Block Size: ", b)

# Number of priority groups
PGROUP_NUM=4

#feerate range 
def get_boundary(pgroup_num):
    boundaries={2:[1], 4:[1,3,10] , 6:[1,2,4,8,23], 8:[1,2,4,7,13,23,50]}
    return boundaries.get(pgroup_num,[1,3,10])

WAIT_BLOCK_NUM_UPPER_BOUNDARY = get_boundary(PGROUP_NUM)
print(WAIT_BLOCK_NUM_UPPER_BOUNDARY)
    

Block Size:  2266.0
[1, 3, 10]


## Priority Groups Setup - feerate range calculation 

In [5]:
length=len(WAIT_BLOCK_NUM_UPPER_BOUNDARY)

# this function returns a list with the feerate boundaries for each priority group
def calculate_feerate_for_priority_groups(data, upper_boundary):
    feerate_range = []
    feerate_range.append(data['feerate'].max())
    transaction_counts=0
    for i in range(length):
        transaction_counts = len(data[data['waiting_block_num']<=upper_boundary[i]])
        boundary = data.feerate.nlargest(transaction_counts).iloc[-1]
        feerate_range.append(boundary)
    feerate_range.append(0)
    return feerate_range
            

feerate_range=calculate_feerate_for_priority_groups(data, WAIT_BLOCK_NUM_UPPER_BOUNDARY)
print("Feerate boundaries: ",feerate_range)


Feerate boundaries:  [52380.95238095238, 30.177140029688267, 18.529411764705888, 5.0, 0]


## Parameters - fixed

In [6]:
# lambda calculation - arrival rate 
total_trans_num = len(data.index)
print("Total number of transactions: ", total_trans_num)

data_sort_by_receive = data.sort_values(by='received_time')
first_arrive_time=float(data['received_time'].min())
last_arrive_time=float(data['received_time'].max())
q_lambda=float(total_trans_num)/(last_arrive_time-first_arrive_time)
mean_interarrival_time=1/q_lambda
print("lambda is: ", q_lambda)

# mu - service rate
service_time_mean=600
mu = 1/service_time_mean
print("Mu is: ", mu)

# total simulation time (seconds)
simulation_time = last_arrive_time-first_arrive_time
print("Total simulation time is: ", simulation_time)

# mempool capacity
trans_size=535
mempool_size=300*1000000
capacity = mempool_size/trans_size
print("Mempool capacity is ", capacity, " transactions")


# store feerates in a list to feed in the model later 
feerates=data.feerate.tolist()

# initiate results 
results=[]


Total number of transactions:  3389593
lambda is:  3.083289746694162
Mu is:  0.0016666666666666668
Total simulation time is:  1099343.0
Mempool capacity is  560747.6635514018  transactions


# M/Mb/1 Queueing Model with Priority Groups Based on Transaction Fees

The data structure deque is selected becaused deque provides an O(1) time complexity for append and pop operations, so that system drlay can be minimised. 

In [7]:
# create named tuple to store single transaction information 
transaction=nt('transaction',['received_time','feerate','confirm_time','enter_block_height','confirmed_block_height'])

#initialise the groups 
deques=[]
for n in range(len(feerate_range)-1):
    queue=deque()
    deques.append(queue)

#t=round(time.time())
t=data.received_time.min()  
next_arrival=t
next_confirm=next_arrival+round(np.random.exponential(service_time_mean))
block_height=data.enter_block_height.min()


## place holder for checking mempool size - not needed for given dataset
# allocate transaction to different priority groups and populating the arrival time  

i=0    
while next_confirm <= t + simulation_time:
    while (next_arrival < next_confirm and i<len(feerates)):  # transactions come in before the next batch
        feerate = feerates[i]
        for k in range(len(deques)):
            if (feerate>feerate_range[k+1]) & (feerate<=feerate_range[k]):  
                deques[k].append(transaction(next_arrival,feerate,0,block_height,0))      
        next_arrival += round(np.random.exponential(1/q_lambda))
        i+=1
        
    # batch processing with size b
    j=0
    block_height = block_height + 1
    for q in deques:
        while q and j<=b:
            # - using popleft() to delete element from left end 
            q[0] = q[0]._replace(confirm_time=next_confirm)
            q[0] = q[0]._replace(confirmed_block_height=block_height)
            confirmed = q.popleft()
            results.append(confirmed)
            j+=1
   
    # calculate next confirmation time 
    next_confirm += round(np.random.exponential(service_time_mean))

for q in deques:
    q.clear()
       
results_df=pd.DataFrame(results,columns=transaction._fields)
results_df=results_df.apply(pd.to_numeric)
results_df['waiting_time']=results_df['confirm_time']-results_df['received_time']
results_df['waited_block_num']=results_df['confirmed_block_height']-results_df['enter_block_height']
results_df.head(50)


,received_time,feerate,confirm_time,enter_block_height,confirmed_block_height,waiting_time,waited_block_num
0,1583756924,6.345106,1583757225,620943,620944,301,1
1,1583756926,7.607251,1583757225,620943,620944,299,1
2,1583756926,5.017784,1583757225,620943,620944,299,1
3,1583756931,5.031142,1583757225,620943,620944,294,1
4,1583756931,6.465054,1583757225,620943,620944,294,1
5,1583756931,7.274534,1583757225,620943,620944,294,1
6,1583756931,6.223325,1583757225,620943,620944,294,1
7,1583756931,6.310757,1583757225,620943,620944,294,1
8,1583756938,8.039024,1583757225,620943,620944,287,1
9,1583756938,8.019465,1583757225,620943,620944,287,1


In [8]:
#calculate average wait time 

def calculate_mean_waiting_time(results_df, feerate_range):
    waiting_times=[]
    for i in range(len(feerate_range)):
        if i<len(feerate_range)-1:
            df=results_df[results_df['feerate'].between(feerate_range[i+1],feerate_range[i],inclusive="right")]
            waiting_time_mean=df['waiting_time'].mean()
            waiting_times.append(waiting_time_mean)
    return waiting_times

waiting_times_estimate=calculate_mean_waiting_time(results_df, feerate_range)
waiting_times_actual=calculate_mean_waiting_time(data, feerate_range)

print("mean waiting times for each priority group (from high to low) are: ", waiting_times_estimate)
print("mean waiting times for each priority group (from high to low) in raw data are: ", waiting_times_actual)        
   

mean waiting times for each priority group (from high to low) are:  [4944.838115675023, 56251.89236406314, 210801.09150801142, 435141.643300995]
mean waiting times for each priority group (from high to low) in raw data are:  [2473.8478372021177, 6085.367607055301, 10045.46922378598, 11979.932554726369]
